# Vectorization and `jax.vmap` in JAX

## Introduction to Vectorization

Vectorization is a technique used in numerical computing to perform operations on entire arrays or batches of data simultaneously, leveraging the inherent parallelism of modern hardware. By operating on vectors or arrays rather than individual elements, vectorized code can achieve significant performance improvements, especially when dealing with large datasets or computational tasks. In JAX, vectorization plays a crucial role in optimizing computations for efficiency and scalability.

Writing fast JAX code requires shifting repetitive tasks from loops to array processing operations, so that the JAX compiler can easily understand the whole operation and generate more efficient machine code.

This procedure is called **vectorization** or **array programming**, and will be familiar to anyone who has used NumPy.

In most ways, vectorization is the same in JAX as it is in NumPy.

In [5]:
import jax.numpy as jnp
import jax
import numpy as np

## Writing Vectorized Code in JAX

Vectorized code in JAX can be written using JAX's NumPy-like API, which supports operations on arrays and tensors. By expressing computations in terms of array operations, such as element-wise arithmetic, matrix multiplication, and broadcasting, developers can harness the full potential of vectorization in JAX.

Let's illustrate this with a simple example

In [2]:
# Define a vectorized computation to calculate the element-wise square of an array
def square_elements(x):
    return x ** 2

# Apply the vectorized computation to an array
input_array = jnp.array([1, 2, 3, 4, 5])
result_array = square_elements(input_array)
print("Squared elements:", result_array)

Squared elements: [ 1  4  9 16 25]


But there are also some differences, which we highlight here.

As a running example, consider the function

$$
f(x,y) = \frac{\cos(x^2 + y^2)}{1 + x^2 + y^2}
$$

Suppose that we want to evaluate this function on a square grid of $ x $ and $ y $ points and then plot it.

To clarify, here is the slow `for` loop version.

In [6]:
@jax.jit
def f(x, y):
    return jnp.cos(x**2 + y**2) / (1 + x**2 + y**2)

n = 80
x = jnp.linspace(-2, 2, n)
y = x

z_loops = np.empty((n, n))

In [7]:
%%time

for i in range(n):
    for j in range(n):
        z_loops[i, j] = f(x[i], y[j])

CPU times: user 1.17 s, sys: 7.38 ms, total: 1.18 s
Wall time: 1.18 s


Even for this very small grid, the run time is extremely slow.

(Notice that we used a NumPy array for `z_loops` because we wanted to write to it.)

OK, so how can we do the same operation in vectorized form?

If you are new to vectorization, you might guess that we can simply write

In [8]:
z_bad = f(x, y)

But this gives us the wrong result because JAX doesn’t understand the nested for loop.

In [9]:
z_bad.shape

(80,)

Here is what we actually wanted:

In [10]:
z_loops.shape

(80, 80)

To get the right shape and the correct nested for loop calculation, we can use a `meshgrid` operation designed for this purpose:

In [11]:
x_mesh, y_mesh = jnp.meshgrid(x, y)

Now we get what we want and the execution time is very fast.

In [12]:
%%time

z_mesh = f(x_mesh, y_mesh) 

CPU times: user 33.6 ms, sys: 2.76 ms, total: 36.3 ms
Wall time: 33.1 ms


In [13]:
%%time

z_mesh = f(x_mesh, y_mesh) 

CPU times: user 533 µs, sys: 85 µs, total: 618 µs
Wall time: 458 µs


Let’s confirm that we got the right answer.

In [14]:
jnp.allclose(z_mesh, z_loops)

Array(True, dtype=bool)

Now we can set up a serious grid and run the same calculation (on the larger grid) in a short amount of time.

In [15]:
n = 6000
x = jnp.linspace(-2, 2, n)
y = x
x_mesh, y_mesh = jnp.meshgrid(x, y)

In [17]:
%%time

z_mesh = f(x_mesh, y_mesh) 

CPU times: user 217 ms, sys: 27.4 ms, total: 244 ms
Wall time: 86.2 ms


In [18]:
%%time

z_mesh = f(x_mesh, y_mesh) 

CPU times: user 217 ms, sys: 24.7 ms, total: 241 ms
Wall time: 84.8 ms


But there is one problem here: the mesh grids use a lot of memory.

In [19]:
x_mesh.nbytes + y_mesh.nbytes

288000000

By comparison, the flat array `x` is just

In [20]:
x.nbytes  # and y is just a pointer to x

24000

This extra memory usage can be a big problem in actual research calculations.

## Using `jax.vmap`

So let’s try a different approach using [jax.vmap](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html)

### Converting loops to `jax.vmap`

The process of converting loops to `jax.vmap` is simple. You take the inner most loop
and vectorized your code over this loop and start forming a nested sequence of `jax.vmap`
from inner most loop to outer most.

Here in our example, first we vectorize `f` in `y` because `y` is in the inner loop.

In [26]:
f_vec_y = jax.vmap(f, in_axes=(None, 0))  

In the line above, `(None, 0)` indicates that we are vectorizing in the second argument, which is `y`.

Next, we vectorize in the first argument, which is `x` and used in the outer loop using
`f_vec_y`.

In [27]:
f_vec = jax.vmap(f_vec_y, in_axes=(0, None))

With this construction, we can now call the function $ f $ on flat (low memory) arrays.

In [23]:
%%time

z_vmap = f_vec(x, y)

CPU times: user 174 ms, sys: 19.2 ms, total: 194 ms
Wall time: 88.6 ms


In [24]:
%%time

z_vmap = f_vec(x, y)

CPU times: user 204 ms, sys: 27.6 ms, total: 232 ms
Wall time: 83.3 ms


In [29]:
z_vmap.shape

(6000, 6000)

The execution time is essentially the same as the mesh operation but we are using much less memory.

And we produce the correct answer:

In [25]:
jnp.allclose(z_vmap, z_mesh)

Array(True, dtype=bool)

### Benefits of Vectorization with jax.vmap

Vectorizing computations with `jax.vmap` offers several benefits:

- **Improved Performance**: Vectorized code can leverage hardware parallelism for faster execution, especially on accelerators like GPUs and TPUs.
- **Simplified Code**: By replacing nested loops with array operations, vectorized code tends to be more concise, readable, and maintainable.
- **Scalability**: Vectorization enables scaling computations to larger datasets or batch sizes without significant performance degradation, making it suitable for machine learning tasks and scientific computing.
- **Nested Vmaps**: Once can easily replace a sequence of python loops using a sequence of `jax.vmap` and JAX handles all the optimizations.